## Techniche - Topic Model

In [1]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel
from gensim.test.utils import common_dictionary, datapath, temporary_file

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re

from topic_model import tokenize_docs

import os, re
from smart_open import smart_open

import matplotlib.pyplot as plt
%matplotlib inline
import pyLDAvis
import pyLDAvis.gensim
from pprint import pprint

/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
np.random.seed(3)

In [3]:
# uncomment to download stop words from nltk
# nltk.download('stopwords')
# nltk.download('punkt')

### Import Data

#### Import data from PatentsView API

In [132]:
# pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 50
pd.set_option('display.max_rows', 50)

# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

# build query
query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},{"_text_phrase":{"patent_abstract":"natural language"}}]}
fields=pat_fields
options={"per_page":2500}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
resp = requests.get(endpoint_url, params=params)
results = resp.json()

#### structure data

In [172]:
# extract metadata from response
print("status code:", resp.status_code,';', "reason:", resp.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data_resp = results['patents']
# data_resp[0]
raw_df = pd.DataFrame(data_resp)
raw_df.head(3)

status code: 200 ; reason: OK
total_patent_count: 2482 ; patents_per_page: 2482


,IPCs,application_citations,applications,assignees,cited_patents,citedby_patents,cpcs,detail_desc_length,examiners,foreign_priority,gov_interests,inventors,lawyers,nbers,patent_abstract,patent_average_processing_time,patent_date,patent_firstnamed_assignee_city,patent_firstnamed_assignee_country,patent_firstnamed_assignee_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_state,patent_firstnamed_inventor_city,patent_firstnamed_inventor_country,patent_firstnamed_inventor_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_state,patent_kind,patent_num_cited_by_us_patents,patent_num_claims,patent_num_combined_citations,patent_num_foreign_citations,patent_num_us_application_citations,patent_num_us_patent_citations,patent_number,patent_processing_time,patent_title,patent_type,patent_year,pct_data,rawinventors,uspcs,wipos
0,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020077823', 'ap...","[{'app_country': 'US', 'app_date': '2013-07-26...","[{'assignee_city': 'Burlington', 'assignee_cou...",[{'cited_patent_category': 'cited by examiner'...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",11570,"[{'examiner_first_name': 'Michael N', 'examine...","[{'forprior_country': None, 'forprior_date': N...","[{'govint_contract_award_number': None, 'govin...","[{'inventor_city': 'Newton', 'inventor_country...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...",Designing a natural language understanding (NL...,None,2019-03-12,Burlington,US,org_ID497r4tFbCIaMBjGAST,42.5047,42.5047|-71.1961,-71.1961,MA,Newton,US,7788103-1,42.3369,42.3369|-71.2097,-71.2097,MA,B2,0,19,31,0,26,5,10229106,2055,Initializing a workspace for building a natura...,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Jeffrey N.', 'raw...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
1,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020138265', 'ap...","[{'app_country': 'US', 'app_date': '2017-09-11...","[{'assignee_city': 'Mountain View', 'assignee_...",[{'cited_patent_category': 'cited by applicant...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",28118,"[{'examiner_first_name': 'Shreyans A', 'examin...","[{'forprior_country': None, 'forprior_date': N...","[{'govint_contract_award_number': None, 'govin...","[{'inventor_city': 'Adliswil', 'inventor_count...","[{'lawyer_first_name': None, 'lawyer_first_see...","[{'nber_category_id': None, 'nber_category_tit...","Methods, systems, and apparatus, including com...",None,2019-03-12,Mountain View,US,org_p6ofWD2xFNSnyYkj6wpA,37.3861,37.3861|-122.0828,-122.083,CA,Adliswil,CH,8352247-1,47.3119,47.3119|8.5287,8.5287,None,B1,0,20,15,0,7,8,10229109,547,Allowing spelling of arbitrary words,utility,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Evgeny A.', 'rawi...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
2,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2001/20010029455', 'ap...","[{'app_country': 'US', 'app_date': '2016-09-28...","[{'assignee_city': 'Seattle', 'assignee_countr...",[{'cited_patent_category': 'cited by applicant...,"[{'citedby_patent_category': None, 'citedby_pa...","[{'cpc_category': None, 'cpc_first_seen_date':...",119654,"[{'examiner_first_name': 'Jialong', 'examiner_...","[{'forprior_country': None, 'forprior_date': N...","[{'govint_contract_award_number': None, 'govin...","[{'inventor_city': 'Seattle', 'inventor_countr...","[{'lawyer_first_name': None, 'lawyer_first_see...","[

#### Subset dataframe

In [165]:
# subset dataframe - comment/uncomment to include fields
df = raw_df[['patent_number', 
         'patent_date', 
         'patent_title',
         'patent_abstract', 
         'patent_firstnamed_assignee_id',
         'patent_firstnamed_assignee_location_id',
         'patent_firstnamed_assignee_latitude',
         'patent_firstnamed_assignee_longitude',
         'patent_firstnamed_assignee_city',
         'patent_firstnamed_assignee_state',
         'patent_firstnamed_assignee_country', 
         'patent_firstnamed_inventor_id',
         'patent_firstnamed_inventor_location_id',
         'patent_firstnamed_inventor_latitude',
         'patent_firstnamed_inventor_longitude',
         'patent_firstnamed_inventor_city',
         'patent_firstnamed_inventor_state',
         'patent_firstnamed_inventor_country',
         'patent_year', 
         'patent_type', 
         'patent_kind',
         'inventors'
#          'patent_processing_time', 
#          'patent_num_us_application_citations', 
#          'patent_num_us_patent_citations', 
#          'patent_num_foreign_citations', 
#          'patent_num_combined_citations', 
#          'patent_num_claims', 
#          'patent_num_cited_by_us_patents',
#          'detail_desc_length'
            ]]

In [166]:
df.head(3)

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,patent_firstnamed_assignee_country,patent_firstnamed_inventor_id,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors
0,10229106,2019-03-12,Initializing a workspace for building a natura...,Designing a natural language understanding (NL...,org_ID497r4tFbCIaMBjGAST,42.5047|-71.1961,42.5047,-71.1961,Burlington,MA,US,7788103-1,42.3369|-71.2097,42.3369,-71.2097,Newton,MA,US,2019,utility,B2,"[{'inventor_city': 'Newton', 'inventor_country..."
1,10229109,2019-03-12,Allowing spelling of arbitrary words,"Methods, systems, and apparatus, including com...",org_p6ofWD2xFNSnyYkj6wpA,37.3861|-122.0828,37.3861,-122.083,Mountain View,CA,US,8352247-1,47.3119|8.5287,47.3119,8.5287,Adliswil,None,CH,2019,utility,B1,"[{'inventor_city': 'Adliswil', 'inventor_count..."
2,10229113,2019-03-12,Leveraging content dimensions during the trans...,A content management system (CMS) and a transl...,org_Vbc6obpnxWM42d0HjlXY,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,US,9177341-1,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,US,2019,utility,B1,"[{'inventor_city': 'Seattle', 'inventor_countr..."


#### Explore data

In [175]:
# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

561

#### Create new column

In [ ]:
# create new column that combines the patent title and the patent abstract columns into a single string
df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

In [174]:
#### TODO (Lee) Partition data into train and test sets

In [182]:
len(df)

2482

In [183]:
df.head(3)

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,patent_firstnamed_assignee_country,patent_firstnamed_inventor_id,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors,patent_title_abstract
0,10229106,2019-03-12,Initializing a workspace for building a natura...,Designing a natural language understanding (NL...,org_ID497r4tFbCIaMBjGAST,42.5047|-71.1961,42.5047,-71.1961,Burlington,MA,US,7788103-1,42.3369|-71.2097,42.3369,-71.2097,Newton,MA,US,2019,utility,B2,"[{'inventor_city': 'Newton', 'inventor_country...",Initializing a workspace for building a natura...
1,10229109,2019-03-12,Allowing spelling of arbitrary words,"Methods, systems, and apparatus, including com...",org_p6ofWD2xFNSnyYkj6wpA,37.3861|-122.0828,37.3861,-122.083,Mountain View,CA,US,8352247-1,47.3119|8.5287,47.3119,8.5287,Adliswil,None,CH,2019,utility,B1,"[{'inventor_city': 'Adliswil', 'inventor_count...","Allowing spelling of arbitrary words Methods, ..."
2,10229113,2019-03-12,Leveraging content dimensions during the trans...,A content management system (CMS) and a transl...,org_Vbc6obpnxWM42d0HjlXY,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,US,9177341-1,47.6064|-122.3308,47.6064,-122.331,Seattle,WA,US,2019,utility,B1,"[{'inventor_city': 'Seattle', 'inventor_countr...",Leveraging content dimensions during the trans...


In [187]:
len(df.patent_number.unique())

2482

In [226]:
df.sort_values(by=['patent_date'])

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,patent_firstnamed_assignee_location_id,patent_firstnamed_assignee_latitude,patent_firstnamed_assignee_longitude,patent_firstnamed_assignee_city,patent_firstnamed_assignee_state,patent_firstnamed_assignee_country,patent_firstnamed_inventor_id,patent_firstnamed_inventor_location_id,patent_firstnamed_inventor_latitude,patent_firstnamed_inventor_longitude,patent_firstnamed_inventor_city,patent_firstnamed_inventor_state,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors,patent_title_abstract
2481,3980994,1976-09-14,Text editing and display system having text in...,A natural language text editing and display sy...,org_1UVZxxNbuUPJkuDfvvpa,42.4906|-71.2767,42.4906,-71.2767,Bedford,MA,US,3980994-1,42.6583|-71.1375,42.6583,-71.1375,Andover,MA,US,1976,utility,A,"[{'inventor_city': 'Andover', 'inventor_countr...",Text editing and display system having text in...
2480,4057849,1977-11-08,Text editing and display system,A text-editing and display system for editing ...,org_1UVZxxNbuUPJkuDfvvpa,42.4906|-71.2767,42.4906,-71.2767,Bedford,MA,US,3980994-1,42.6583|-71.1375,42.6583,-71.1375,Andover,MA,US,1977,utility,A,"[{'inventor_city': 'Andover', 'inventor_countr...",Text editing and display system A text-editing...
2479,4502128,1985-02-26,Translation between natural languages,An input sentence described by a first natural...,org_70D1lR89kQnFiCFdJ6s5,35.685|139.7514,35.685,139.751,Tokyo,None,JP,4502128-1,35.4437|139.638,35.4437,139.638,Yokohama,None,JP,1985,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Translation between natural languages An input...
2478,4586160,1986-04-29,Method and apparatus for analyzing the syntact...,An automatic syntax analyzing method is applie...,org_hDziASDpeFilN1JsnVK0,35.5298|139.7024,35.5298,139.702,Kawasaki,None,JP,4586160-1,35.4437|139.638,35.4437,139.638,Yokohama,None,JP,1986,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Method and apparatus for analyzing the syntact...
2477,4599612,1986-07-08,Displaying and correcting method for machine t...,In a system wherein a first text in a first na...,org_70D1lR89kQnFiCFdJ6s5,35.685|139.7514,35.685,139.751,Tokyo,None,JP,4599612-1,34.4783|133.9333,34.4783,133.933,Tama,None,JP,1986,utility,A,"[{'inventor_city': 'Fujisawa', 'inventor_count...",Displaying and correcting method for machine t...
2476,4638445,1987-01-20,Autonomous mobile robot,A vision system for a mobile robot employs at ...,None,None,None,None,None,None,None,4638445-1,42.1875|-71.3069,42.1875,-71.3069,Medfield,MA,US,1987,utility,A,"[{'inventor_city': 'Medfield', 'inventor_count...",Autonomous mobile robot A vision system for a ...
2475,4661924,1987-04-28,Multiple-parts-of-speech disambiguating method...,A machine translation system comprises input m...,org_70D1lR89kQnFiCFdJ6s5,35.685|139.7514,35.685,139.751,Tokyo,None,JP,4661924-1,35.685|139.7514,35.685,139.751,Tokyo,None,JP,1987,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Multiple-parts-of-speech disambiguating method...
2474,4688195,1987-08-18,Natural-language interface generating system,A system for interactively generating a natura...,org_C6OV0HJ5yTbw8A4Wv6Ag,32.7833|-96.8,32.7833,-96.8,Dallas,TX,US,4688195-1,33.0197|-96.6986,33.0197,-96.6986,Plano,TX,US,1987,utility,A,"[{'inventor_city': 'Plano', 'inventor_country'...",Natural-language interface generating system A...
2473,4689737,1987-08-25,Integrated environment computer system control...,A computer system including text input and dis...,None,None,None,None,None,None,None,4689737-1,34.1511|-118.4483,34.1511,-118.448,Sherman Oaks,CA,US,1987,utility,A,"[{'inventor_city': 'Sherman Oaks', 'inventor_c...",Integrated environment computer system control...
2472,4695975,1987-09-22,Multi-image communications system,A device for automatic translation of natural ...,org_rjvvHMFnrkq3zHsiW0ej,40.7142|-74.0064,40.7142,-74.0064,New York,NY,US,4695975-1,4

In [222]:
dat2 = df.patent_title_abstract.tolist()

In [223]:
dat2

['Initializing a workspace for building a natural language understanding system Designing a natural language understanding (NLU) model for an application from scratch can be difficult for non-experts. A system can simplify the design process by providing an interface allowing a designer to input example usage sentences and build an NLU model based on presented matches to those example sentences. In one embodiment, a method for initializing a workspace for building an NLU system includes parsing a sample sentence to select at least one candidate stub grammar from among multiple candidate stub grammars. The method can include presenting, to a user, respective representations of the candidate stub grammars selected by the parsing of the sample sentence. The method can include enabling the user to choose one of the respective representations of the candidate stub grammars. The method can include adding to the workspace a stub grammar corresponding to the representation of the candidate stu

### Pre-process text data

#### Tokenize

In [12]:
# tokenize documents

def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

tokenized_docs = tokenize_docs(data)

#### Clean punctuation

In [13]:
# clean punctuation
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [15]:
data = clean_docs(tokenized_docs)
data[0]

['Translation',
 'between',
 'natural',
 'languages',
 'An',
 'input',
 'sentence',
 'described',
 'by',
 'a',
 'first',
 'natural',
 'language',
 'is',
 'sectioned',
 'into',
 'individual',
 'words',
 'Parts',
 'of',
 'speech',
 'corresponding',
 'to',
 'the',
 'individual',
 'words',
 'are',
 'retrieved',
 'from',
 'a',
 'lexical',
 'word',
 'storage',
 'whereby',
 'the',
 'input',
 'sentence',
 'is',
 'described',
 'by',
 'a',
 'corresponding',
 'string',
 'of',
 'the',
 'as',
 'retrieved',
 'A',
 'translation',
 'pattern',
 'table',
 'is',
 'previously',
 'prepared',
 'which',
 'defines',
 'correspondence',
 'between',
 'patterns',
 'of',
 'strings',
 'of',
 'for',
 'the',
 'first',
 'natural',
 'language',
 'and',
 'those',
 'for',
 'a',
 'second',
 'natural',
 'language',
 'by',
 'which',
 'corresponding',
 'output',
 'sentence',
 'is',
 'to',
 'be',
 'described',
 'By',
 'referring',
 'to',
 'the',
 'translation',
 'pattern',
 'table',
 'the',
 'string',
 'of',
 'the',
 'of',
 '

#### Convert to lowercase

In [17]:
# convert to lowercase
def lower_words(docs):
    lowered_words = []
    for doc in docs:
        lowered_words.append([word.lower() for word in doc])
    return lowered_words

lower_words(data)[0]

['translation',
 'between',
 'natural',
 'languages',
 'an',
 'input',
 'sentence',
 'described',
 'by',
 'a',
 'first',
 'natural',
 'language',
 'is',
 'sectioned',
 'into',
 'individual',
 'words',
 'parts',
 'of',
 'speech',
 'corresponding',
 'to',
 'the',
 'individual',
 'words',
 'are',
 'retrieved',
 'from',
 'a',
 'lexical',
 'word',
 'storage',
 'whereby',
 'the',
 'input',
 'sentence',
 'is',
 'described',
 'by',
 'a',
 'corresponding',
 'string',
 'of',
 'the',
 'as',
 'retrieved',
 'a',
 'translation',
 'pattern',
 'table',
 'is',
 'previously',
 'prepared',
 'which',
 'defines',
 'correspondence',
 'between',
 'patterns',
 'of',
 'strings',
 'of',
 'for',
 'the',
 'first',
 'natural',
 'language',
 'and',
 'those',
 'for',
 'a',
 'second',
 'natural',
 'language',
 'by',
 'which',
 'corresponding',
 'output',
 'sentence',
 'is',
 'to',
 'be',
 'described',
 'by',
 'referring',
 'to',
 'the',
 'translation',
 'pattern',
 'table',
 'the',
 'string',
 'of',
 'the',
 'of',
 '

#### Clean stopwords

In [19]:
# clean stopwords

stop_words = stopwords.words('english')

def remove_stopwords(clean_docs):
    filtered_docs = []
    for doc in clean_docs:
       filtered_docs.append([word for word in doc if word not in stop_words])
    return filtered_docs

# remove stopwords
cleaned_data = remove_stopwords(data)
cleaned_data[0]
# TODO (Lee) - resolve un-lowered stopwords "A" and "An", 'By', 'The'

['Translation',
 'natural',
 'languages',
 'An',
 'input',
 'sentence',
 'described',
 'first',
 'natural',
 'language',
 'sectioned',
 'individual',
 'words',
 'Parts',
 'speech',
 'corresponding',
 'individual',
 'words',
 'retrieved',
 'lexical',
 'word',
 'storage',
 'whereby',
 'input',
 'sentence',
 'described',
 'corresponding',
 'string',
 'retrieved',
 'A',
 'translation',
 'pattern',
 'table',
 'previously',
 'prepared',
 'defines',
 'correspondence',
 'patterns',
 'strings',
 'first',
 'natural',
 'language',
 'second',
 'natural',
 'language',
 'corresponding',
 'output',
 'sentence',
 'described',
 'By',
 'referring',
 'translation',
 'pattern',
 'table',
 'string',
 'input',
 'sentence',
 'transformed',
 'corresponding',
 'string',
 'second',
 'natural',
 'language',
 'The',
 'output',
 'sentence',
 'described',
 'second',
 'natural',
 'language',
 'generated',
 'sequencing',
 'target',
 'words',
 'accordance',
 'sequential',
 'order',
 'parts',
 'speech',
 'string',
 'pa

#### Create bigrams and trigrams

In [20]:
# train bigram phrases model
bigram_model = Phrases(data, min_count=1, threshold=1)

# train trigram phrases model
trigram_model = Phrases(bigram_model[data], threshold=100)  

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [21]:
# bigrams
def bigrams(docs):
    """create bigrams"""
    return [bigram_model[doc] for doc in docs]

In [22]:
# initialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram_model)
trigram_model = gensim.models.phrases.Phraser(trigram_model)

In [26]:
bigrams(cleaned_data)[0]

['Translation',
 'natural_languages',
 'An_input',
 'sentence_described',
 'first_natural',
 'language',
 'sectioned',
 'individual_words',
 'Parts',
 'speech',
 'corresponding',
 'individual_words',
 'retrieved',
 'lexical',
 'word',
 'storage',
 'whereby',
 'input_sentence',
 'described',
 'corresponding_string',
 'retrieved_A',
 'translation_pattern',
 'table',
 'previously',
 'prepared',
 'defines',
 'correspondence',
 'patterns',
 'strings',
 'first_natural',
 'language',
 'second_natural',
 'language',
 'corresponding',
 'output_sentence',
 'described',
 'By',
 'referring',
 'translation_pattern',
 'table',
 'string',
 'input_sentence',
 'transformed',
 'corresponding_string',
 'second_natural',
 'language',
 'The',
 'output_sentence',
 'described',
 'second_natural',
 'language',
 'generated',
 'sequencing',
 'target_words',
 'accordance',
 'sequential',
 'order',
 'parts',
 'speech',
 'string',
 'pattern',
 'obtained',
 'transformation']

In [24]:
def trigrams(docs):
    """create trigrams"""
    return [trigram_model[bigram_model[doc]] for doc in docs]

In [25]:
trigrams(cleaned_data)

[['Translation',
  'natural_languages',
  'An_input',
  'sentence_described',
  'first_natural',
  'language',
  'sectioned',
  'individual_words',
  'Parts',
  'speech',
  'corresponding',
  'individual_words',
  'retrieved',
  'lexical',
  'word',
  'storage',
  'whereby',
  'input_sentence',
  'described',
  'corresponding_string',
  'retrieved_A',
  'translation_pattern',
  'table',
  'previously',
  'prepared',
  'defines',
  'correspondence',
  'patterns',
  'strings',
  'first_natural',
  'language',
  'second_natural',
  'language',
  'corresponding',
  'output_sentence',
  'described',
  'By',
  'referring',
  'translation_pattern',
  'table',
  'string',
  'input_sentence',
  'transformed',
  'corresponding_string',
  'second_natural',
  'language',
  'The',
  'output_sentence',
  'described',
  'second_natural',
  'language',
  'generated',
  'sequencing',
  'target_words',
  'accordance',
  'sequential',
  'order',
  'parts',
  'speech',
  'string',
  'pattern',
  'obtained

#### Stem and Lemmatize

In [27]:
def lemmatize_docs(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """lemmatize documents"""
    lemmatized_docs = []
    for doc in docs: 
        lemmatized_docs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return lemmatized_docs

In [28]:
# TODO (Lee)

lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
# for doc in cleaned_data:
#     for token in doc:
#         token.lemma_

# uncomment to use
# download english model with "python -m spacy download en"

# for token in doc:
#     print(token, token.lemma, token.lemma_)

# TODO (Lee) - lemmatize_docs(cleaned_data)

#### Create corpus and dictionary

In [159]:
 # build dictionary
id_word = corpora.Dictionary(cleaned_data)

# build corpus
texts = cleaned_data

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_word.doc2bow(text) for text in texts]

In [34]:
# view formatted corpus (term-doc-frequency)
[[(id_word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('A', 1),
  ('An', 1),
  ('By', 1),
  ('Parts', 1),
  ('The', 1),
  ('Translation', 1),
  ('accordance', 1),
  ('correspondence', 1),
  ('corresponding', 4),
  ('defines', 1),
  ('described', 4),
  ('first', 2),
  ('generated', 1),
  ('individual', 2),
  ('input', 3),
  ('language', 5),
  ('languages', 1),
  ('lexical', 1),
  ('natural', 6),
  ('obtained', 1),
  ('order', 1),
  ('output', 2),
  ('parts', 1),
  ('pattern', 3),
  ('patterns', 1),
  ('prepared', 1),
  ('previously', 1),
  ('referring', 1),
  ('retrieved', 2),
  ('second', 3),
  ('sectioned', 1),
  ('sentence', 5),
  ('sequencing', 1),
  ('sequential', 1),
  ('speech', 2),
  ('storage', 1),
  ('string', 4),
  ('strings', 1),
  ('table', 2),
  ('target', 1),
  ('transformation', 1),
  ('transformed', 1),
  ('translation', 2),
  ('whereby', 1),
  ('word', 1),
  ('words', 3)]]

### Model - model #1

In [36]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

In [39]:
# print keywords in 10 topics
pprint(model_lda.print_topics())

[(7,
  '0.128*"social" + 0.088*"network" + 0.043*"parse" + 0.039*"concept" + '
  '0.035*"nodes" + 0.031*"The" + 0.022*"kernel" + 0.020*"hashtags" + '
  '0.017*"node" + 0.016*"outputs"'),
 (5,
  '0.089*"entity" + 0.087*"token" + 0.030*"quality" + 0.029*"disambiguation" + '
  '0.028*"Disclosed" + 0.027*"herein" + 0.019*"tasks" + 0.018*"state" + '
  '0.017*"signal" + 0.015*"manually"'),
 (12,
  '0.079*"NLU" + 0.043*"model" + 0.034*"used" + 0.031*"For" + 0.023*"likely" + '
  '0.022*"understanding" + 0.021*"scanned" + 0.019*"grammatical" + '
  '0.017*"using" + 0.017*"accurate"'),
 (23,
  '0.073*"document" + 0.030*"item" + 0.029*"form" + 0.029*"type" + '
  '0.023*"list" + 0.023*"statements" + 0.021*"items" + 0.021*"The" + '
  '0.019*"defined" + 0.018*"analysis"'),
 (0,
  '0.093*"message" + 0.072*"component" + 0.047*"messages" + 0.029*"string" + '
  '0.025*"metadata" + 0.022*"mode" + 0.021*"statement" + 0.017*"processors" + '
  '0.017*"generate" + 0.016*"character"'),
 (19,
  '0.080*"word" + 

In [45]:
# print top 10 keywords that comprise topic with index of 0
pprint(model_lda.print_topic(24))
# the most import keywords, and the respective weight, that form topic 0 are

('0.093*"question" + 0.060*"answer" + 0.041*"The" + 0.032*"user" + '
 '0.030*"approach" + 0.028*"system" + 0.027*"questions" + 0.023*"includes" + '
 '0.023*"based" + 0.020*"potential"')


In [46]:
# print top 10 keywords that comprise topic with index of 1
pprint(model_lda.print_topic(1))

('0.108*"content" + 0.059*"natural" + 0.058*"language" + 0.043*"model" + '
 '0.037*"using" + 0.026*"set" + 0.026*"processing" + 0.023*"The" + '
 '0.023*"information" + 0.021*"features"')


In [47]:
# TODO (Lee) - infer topic from keywords?

### Evaluate - model #1

In [50]:
# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
perplexity

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

-6.800299187946384

In [48]:
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=data, dictionary=id_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

0.38964237141847535

In [51]:
# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

[0.4106267971162076,
 0.31722446067803695,
 0.5478579066803337,
 0.4132573830140969,
 0.32067875597825285,
 0.5294790508063428,
 0.49700275912588465,
 0.4066938157686664,
 0.26671360789845366,
 0.25329437453386394,
 0.5161361938345662,
 0.5091895950376919,
 0.35429888480919225,
 0.28219213238278434,
 0.47898701912650177,
 0.36814798745852895,
 0.22946963311075047,
 0.3229132041847006,
 0.29459466613983126,
 0.40453055915491465,
 0.363053418841915,
 0.24193727209161947,
 0.6471154823395759,
 0.45047726778912206,
 0.31518705756004844]

In [54]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus, id_word)
viz_topics_1
# TODO (Lee) - salient vs relevant terms in pyLDA ?

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.275342 -0.017475       1        1  16.970181
20    -0.184748  0.154687       2        1   9.940348
1     -0.166868  0.123700       3        1   5.275103
17    -0.158460 -0.146844       4        1   5.268406
4     -0.196499  0.060736       5        1   5.197737
8     -0.087633  0.115791       6        1   5.172447
16    -0.066838  0.115062       7        1   4.817734
3     -0.067673 -0.119506       8        1   4.182312
24    -0.118902 -0.144861       9        1   3.922304
2      0.148475  0.009011      10        1   3.564424
9     -0.028773  0.015363      11        1   3.319834
22     0.009257 -0.230990      12        1   3.215718
15     0.075009 -0.138836      13        1   3.036491
21    -0.025356  0.064032      14        1   3.008785
18    -0.021549 -0.208762      15        1   2.991052
6      0.091142  0.015996      16        1   2.663868
23     0.062774  0.083576      17        1   2.197709
0      0.087411  0.077808      18        1   2.110543
10     0.119236  0.047846      19        1   2.098179
11     0.161343  0.030224      20        1   2.071341
13     0.091502  0.008124      21        1   2.062785
19     0.164545  0.037829      22        1   2.017935
7      0.128687  0.003770      23        1   1.778670
12     0.080378  0.050555      24        1   1.687973
5      0.178881 -0.006834      25        1   1.428115, topic_info=     Category         Freq           Term        Total  loglift  logprob
term                                                                    
150   Default   721.000000           data   721.000000  30.0000  30.0000
538   Default   606.000000          query   606.000000  29.0000  29.0000
531   Default  1017.000000            may  1017.000000  28.0000  28.0000
134   Default  1072.000000           user  1072.000000  27.0000  27.0000
113   Default   420.000000        content   420.000000  26.0000  26.0000
668   Default   356.000000         search   356.000000  25.0000  25.0000
15    Default  2175.000000       language  2175.000000  24.0000  24.0000
1106  Default   290.000000         dialog   290.000000  23.0000  23.0000
18    Default  1890.000000        natural  1890.000000  22.0000  22.0000
567   Default   269.000000       question   269.000000  21.0000  21.0000
11    Default   485.000000          first   485.000000  20.0000  20.0000
69    Default   503.000000           text   503.000000  19.0000  19.0000
34    Default   292.000000         speech   292.000000  18.0000  18.0000
68    Default  1037.000000         system  1037.000000  17.0000  17.0000
223   Default   863.000000            one   863.000000  16.0000  16.0000
3671  Default   168.000000         social   168.000000  15.0000  15.0000
1437  Default   174.000000        product   174.000000  14.0000  14.0000
461   Default   424.000000            set   424.000000  13.0000  13.0000
122   Default   627.000000    information   627.000000  12.0000  12.0000
14    Default   551.000000          input   551.000000  11.0000  11.0000
289   Default   682.000000     processing   682.000000  10.0000  10.0000
592   Default   306.000000          model   306.000000   9.0000   9.0000
992   Default   166.000000          voice   166.000000   8.0000   8.0000
603   Default   180.000000      utterance   180.000000   7.0000   7.0000
31    Default   169.000000       sentence   169.000000   6.0000   6.0000
...       ...          ...            ...          ...      ...      ...
4355  Topic25     8.576556      assurance     9.498975   4.1467  -4.8062
3019  Topic25     8.299238   participants     9.221657   4.1434  -4.8391
177   Topic25     7.932952    Furthermore     8.855371   4.1388  -4.8842
2453  Topic25     7.720916     disclosure     8.643335   4.1360  -4.9113
3440  Topic25     7.675465          forum     8.597884   4.1353  -4.9172
1054  Topic25     7.512743       boundary     8.435163   4.1330  -4.9386
1643  Topic25    

### Model 2-  Mallet model

In [55]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update this path
path_mallet = 'data/mallet-2.0.8/bin/mallet'

In [56]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=25, id2word=id_word)

In [57]:
# topics
pprint(model_2.show_topics(formatted=False))

[(16,
  [('natural', 0.06486254295532647),
   ('language', 0.05970790378006873),
   ('selected', 0.05197594501718213),
   ('module', 0.04252577319587629),
   ('portion', 0.03865979381443299),
   ('statistical', 0.03178694158075601),
   ('configured', 0.028350515463917526),
   ('engine', 0.027061855670103094),
   ('entry', 0.02620274914089347),
   ('classification', 0.01718213058419244)]),
 (10,
  [('speech', 0.12660485021398002),
   ('model', 0.09807417974322397),
   ('recognition', 0.08345221112696148),
   ('models', 0.043509272467902996),
   ('NLU', 0.03673323823109843),
   ('utterance', 0.03566333808844508),
   ('understanding', 0.029243937232524966),
   ('training', 0.026034236804564907),
   ('processing', 0.01818830242510699),
   ('ASR', 0.014978601997146932)]),
 (8,
  [('user', 0.0837173579109063),
   ('response', 0.051075268817204304),
   ('product', 0.039938556067588324),
   ('service', 0.029185867895545316),
   ('call', 0.024961597542242704),
   ('agent', 0.021121351766513058)

In [58]:
# calculate coherence metric
coherence_model_2 = CoherenceModel(model=model_2, texts=data, dictionary=id_word, coherence='c_v')
coherence_model_2 = coherence_model_2.get_coherence()
coherence_model_2

0.3398511614777415

In [59]:
# TODO (Lee)
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in range(start, limit, step):
#         model = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

# model_list, coherence_values = compute_coherence_values(dictionary=id_word, corpus=corpus, texts=data, start=2, limit=40, step=6)

### Model 3 - Author topic model

In [145]:
# construct inventor-to-doc mapping as df from nested inventors column in json api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number'])
df_inventors = df_inventors[['inventor_id', 'patent_number']]
df_inventors.head(3)

,inventor_id,patent_number
0,7788103-1,10229106
1,8352247-1,10229109
2,8515750-2,10229109


In [151]:
len(df_inventors['inventor_id'].unique())

4039

In [78]:
# #df_inventors.set_index('inventor_id')
# inv_to_doc = dict(zip(df_inventors.inventor_id, df_inventors.patent_number))
# inv_to_doc

{'7788103-1': '8903712',
 '8352247-1': '9514743',
 '8515750-2': '9971758',
 '8849675-1': '9971758',
 '9514743-1': '9514743',
 '9971758-5': '9971758',
 '10223356-1': '10223356',
 '10223356-2': '10223356',
 '10223356-3': '10223356',
 '10223356-5': '10223356',
 '9177341-1': '10223356',
 '9959271-8': '10223356',
 '8281187-1': '10095736',
 '9442919-1': '10002124',
 '9442919-2': '9996526',
 '9442919-3': '10002124',
 '9442919-5': '9996526',
 '8560468-5': '9589060',
 '9037568-4': '9589060',
 '9292545-3': '9589060',
 '9589060-4': '9589060',
 '9589060-5': '9589060',
 '9589060-6': '9589060',
 '10229187-1': '10229189',
 '10229187-2': '10229189',
 '6718519-1': '10229189',
 '7945556-2': '10229189',
 '9280340-1': '10146858',
 '7398209-3': '7398209',
 '8493600-1': '10229678',
 '6804574-3': '9613020',
 '9319838-3': '9613020',
 '9606701-1': '9613020',
 '10229680-1': '10229680',
 '10229680-2': '10229680',
 '5712675-3': '10229687',
 '7475015-5': '9311298',
 '7233733-3': '10225227',
 '7558642-3': '10225227

In [122]:
df_inventors.set_index('inventor_id').T.to_dict('list')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


{'7788103-1': ['8903712'],
 '8352247-1': ['9514743'],
 '8515750-2': ['9971758'],
 '8849675-1': ['9971758'],
 '9514743-1': ['9514743'],
 '9971758-5': ['9971758'],
 '10223356-1': ['10223356'],
 '10223356-2': ['10223356'],
 '10223356-3': ['10223356'],
 '10223356-5': ['10223356'],
 '9177341-1': ['10223356'],
 '9959271-8': ['10223356'],
 '8281187-1': ['10095736'],
 '9442919-1': ['10002124'],
 '9442919-2': ['9996526'],
 '9442919-3': ['10002124'],
 '9442919-5': ['9996526'],
 '8560468-5': ['9589060'],
 '9037568-4': ['9589060'],
 '9292545-3': ['9589060'],
 '9589060-4': ['9589060'],
 '9589060-5': ['9589060'],
 '9589060-6': ['9589060'],
 '10229187-1': ['10229189'],
 '10229187-2': ['10229189'],
 '6718519-1': ['10229189'],
 '7945556-2': ['10229189'],
 '9280340-1': ['10146858'],
 '7398209-3': ['7398209'],
 '8493600-1': ['10229678'],
 '6804574-3': ['9613020'],
 '9319838-3': ['9613020'],
 '9606701-1': ['9613020'],
 '10229680-1': ['10229680'],
 '10229680-2': ['10229680'],
 '5712675-3': ['10229687'],
 '

In [221]:
pat2inv = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
test

{'10002124': ['9442919-1', '9442919-2', '9442919-3', '9442919-5'],
 '10002129': ['9919723-2'],
 '10002188': ['8117606-2', '8881104-1', '9245015-2'],
 '10002201': ['6672422-2', '7343297-3', '7401072-2', 'D529036-3'],
 '10002607': ['10002607-1',
  '10002607-3',
  '10002607-4',
  '10002607-5',
  '10002607-6',
  '10002607-7',
  '10002607-8',
  '9002798-2'],
 '10003559': ['9740579-3', '9916318-3'],
 '10003683': ['10003683-1',
  '10003683-2',
  '10003683-3',
  '10003683-4',
  '10003683-5',
  '10003683-6',
  '10003683-7',
  '10003683-8',
  '10003683-9',
  '6288759-3'],
 '10007658': ['7088873-1', '9817812-1', '9817812-2'],
 '10007659': ['4847805-1', '6823309-4', '9870768-2'],
 '10007660': ['7475015-5', '9047560-3', '9318109-3', '9690776-4'],
 '10009462': ['10009462-1', '10009462-2', '9740765-3'],
 '10013266': ['10013266-1', '10013266-2'],
 '10013404': ['7549132-1', '8214693-3', '8676787-1', '9129213-2'],
 '10013416': ['10013416-1',
  '10013416-10',
  '10013416-11',
  '10013416-12',
  '10013416

In [156]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus,
                         author2doc=inv2doc,
                         id2word=id_word, 
                         num_topics=10)

TypeError: can only concatenate list (not "int") to list

In [ ]:
# gensim example
# mmcorpus.IndexedCorpus
# from gensim.models import AuthorTopicModel
# from gensim.corpora import mmcorpus
# from gensim.test.utils import common_dictionary, datapath, temporary_file

# gensim author2doc format example:
author2doc = {
     'john': [0, 1, 2, 3, 4, 5, 6],
     'jane': [2, 3, 4, 5, 6, 7, 8],
     'jack': [0, 2, 4, 6, 8]
 }
author2doc

# corpus = mmcorpus.MmCorpus(datapath('testcorpus.mm'))
# corpus[0]

In [ ]:
# construct vectors for authors
author_vecs = [model.get_author_topics(author) for author in model.id2author.values()]

#### Load data

In [ ]:
#### Construct a mapping from authors to document IDs

In [ ]:
filenames = [data_dir + 'idx/a' + yr + '.txt' for yr in yrs]  # Using the years defined in previous cell.